# fit silico Horvath probes

Run pipelines

```bash
sbatch cluster/processing/fitting/horvath/fit_horvath_silico_probe1_noise_layer1.sbatch
sbatch cluster/processing/fitting/horvath/fit_horvath_silico_probe1_noise_layer2_3.sbatch
sbatch cluster/processing/fitting/horvath/fit_horvath_silico_probe2_noise_layer2_3.sbatch
sbatch cluster/processing/fitting/horvath/fit_horvath_silico_probe2_noise_layer4.sbatch
sbatch cluster/processing/fitting/horvath/fit_horvath_silico_probe2_noise_layer5.sbatch
sbatch cluster/processing/fitting/horvath/fit_horvath_silico_probe3_noise_layer6.sbatch # 2 hours
````

In [30]:
# import libs
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from matplotlib import pyplot as plt
import numpy as np
import scipy
import spikeinterface.extractors as se 
import os
import spikeinterface as si
import spikeinterface.full as si_full
import spikeinterface.preprocessing as spre
import pandas as pd
from scipy.optimize import minimize
import numpy as np
from concurrent.futures import ProcessPoolExecutor
import copy

# move to PROJECT PATH
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/spikebias/"
os.chdir(PROJ_PATH)

from src.nodes.utils import get_config
from src.nodes.study import amplitude
from src.nodes.prepro import preprocess, trace_fitting
from src.nodes.dataeng.silico import recording


# SETUP PARAMETERS
RND_SEED = 0                # random seed
SFREQ_VIVO = 20000          # sampling frequency
SFREQ_SILICO = 20000        # sampling frequency
WIND_END = 3700             # last segment to calculate mad


# VIVO -------------------
# probe 1
data_conf_vivo_probe_1, param_conf_vivo_probe_1 = get_config(
    "vivo_horvath", "probe_1"
).values() 
RAW_PATH_VIVO_PROBE_1 = data_conf_vivo_probe_1["raw"]
PREP_PATH_VIVO_PROBE_1 = data_conf_vivo_probe_1["preprocessing"]["output"]["trace_file_path"]
CONTACTS_VIVO = np.arange(0,128,1)

# probe 2
data_conf_vivo_probe_2, param_conf_vivo_probe_2 = get_config(
    "vivo_horvath", "probe_1"
).values() 
RAW_PATH_VIVO_PROBE_2 = data_conf_vivo_probe_2["raw"]
PREP_PATH_VIVO_PROBE_2 = data_conf_vivo_probe_2["preprocessing"]["output"]["trace_file_path"]
CONTACTS_SILICO = np.arange(0,128,1)

# probe 3
data_conf_vivo_probe_3, param_conf_vivo_probe_3 = get_config(
    "vivo_horvath", "probe_2"
).values() 
RAW_PATH_VIVO_PROBE_3 = data_conf_vivo_probe_3["raw"]
PREP_PATH_VIVO_PROBE_3 = data_conf_vivo_probe_3["preprocessing"]["output"]["trace_file_path"]
CONTACTS_SILICO = np.arange(0,128,1)


# SILICO -------------------
# probe 1
data_conf_silico_probe_1, param_conf_silico_probe_1 = get_config(
    "silico_horvath", "concatenated/probe_1"
).values()
RAW_PATH_SILICO_PROBE_1 = data_conf_silico_probe_1["dataeng"]["campaign"]["output"]["trace_file_path"]
PREP_PATH_SILICO_PROBE_1 = data_conf_silico_probe_1["preprocessing"]["output"]["trace_file_path"]
MISSING_NOISE_PATH_PROBE_1 = data_conf_silico_probe_1["preprocessing"]["fitting"]["missing_noise_path"]

# probe 2
data_conf_silico_probe_2, param_conf_silico_probe_2 = get_config(
    "silico_horvath", "concatenated/probe_2"
).values()
RAW_PATH_SILICO_PROBE_2 = data_conf_silico_probe_2["dataeng"]["campaign"]["output"]["trace_file_path"]
PREP_PATH_SILICO_PROBE_2 = data_conf_silico_probe_2["preprocessing"]["output"]["trace_file_path"]
MISSING_NOISE_PATH_PROBE_2 = data_conf_silico_probe_2["preprocessing"]["fitting"]["missing_noise_path"]

# probe 3
data_conf_silico_probe_3, param_conf_silico_probe_3 = get_config(
    "silico_horvath", "concatenated/probe_3"
).values()
RAW_PATH_SILICO_PROBE_3 = data_conf_silico_probe_3["dataeng"]["campaign"]["output"]["trace_file_path"]
PREP_PATH_SILICO_PROBE_3 = data_conf_silico_probe_3["preprocessing"]["output"]["trace_file_path"]
MISSING_NOISE_PATH_PROBE_3 = data_conf_silico_probe_3["preprocessing"]["fitting"]["missing_noise_path"]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2023-12-20 16:05:10,361 - root - utils.py - get_config - INFO - Reading experiment config.
2023-12-20 16:05:10,384 - root - utils.py - get_config - INFO - Reading experiment config. - done
2023-12-20 16:05:10,396 - root - utils.py - get_config - INFO - Reading experiment config.
2023-12-20 16:05:10,403 - root - utils.py - get_config - INFO - Reading experiment config. - done
2023-12-20 16:05:10,404 - root - utils.py - get_config - INFO - Reading experiment config.
2023-12-20 16:05:10,411 - root - utils.py - get_config - INFO - Reading experiment config. - done
2023-12-20 16:05:10,412 - root - utils.py - get_config - INFO - Reading experiment config.
2023-12-20 16:05:10,423 - root - utils.py - get_config - INFO - Reading experiment config. - done
2023-12-20 16:05:10,424 - root - utils.py - get_config - INFO - Reading experiment config.
2023-12-20 16:05:10,435 - root - utils.py - get_config - INFO - R

### If data already exists

In [31]:
# if already fitted results exist
missing_noise_probe_1_L1 = np.load(
    MISSING_NOISE_PATH_PROBE_1 + "L1.npy", allow_pickle=True
).item()
missing_noise_probe_1_L2_3 = np.load(
    MISSING_NOISE_PATH_PROBE_1 + "L2_3.npy", allow_pickle=True
).item()
missing_noise_probe_2_L2_3 = np.load(
    MISSING_NOISE_PATH_PROBE_2 + "L2_3.npy", allow_pickle=True
).item()
missing_noise_probe_2_L4 = np.load(
    MISSING_NOISE_PATH_PROBE_2 + "L4.npy", allow_pickle=True
).item()
missing_noise_probe_2_L5 = np.load(
    MISSING_NOISE_PATH_PROBE_2 + "L5.npy", allow_pickle=True
).item()
missing_noise_probe_3_L6 = np.load(
    MISSING_NOISE_PATH_PROBE_3 + "L6.npy", allow_pickle=True
).item()

# store missing noises
layers_missing_noise = (
    missing_noise_probe_1_L1,
    missing_noise_probe_1_L2_3,
    missing_noise_probe_2_L2_3,
    missing_noise_probe_2_L4,
    missing_noise_probe_2_L5,
    missing_noise_probe_3_L6,
)

# print
print(layers_missing_noise)

# sanity check fitted noises
print(missing_noise_probe_1_L1["missing_noise_rms"])
print(missing_noise_probe_1_L2_3["missing_noise_rms"])
print(missing_noise_probe_2_L2_3["missing_noise_rms"])
print(missing_noise_probe_2_L4["missing_noise_rms"])
print(missing_noise_probe_2_L5["missing_noise_rms"])
print(missing_noise_probe_3_L6["missing_noise_rms"])

({'missing_noise_rms': 26.34375, 'layer_sites_ix': [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 80, 81, 82, 83, 84, 85, 86, 87, 88, 113, 114, 115, 116, 117, 118, 119], 'seed': 0}, {'missing_noise_rms': 29.28125, 'layer_sites_ix': [8, 9, 10, 11, 12, 13, 14, 40, 41, 42, 43, 44, 45, 46, 47, 73, 74, 75, 76, 77, 78, 79, 104, 105, 106, 107, 108, 109, 110, 111, 112, 0, 1, 2, 3, 4, 5, 6, 7, 32, 33, 34, 35, 36, 37, 38, 39, 64, 65, 66, 67, 68, 69, 70, 71, 72, 96, 97, 98, 99, 100, 101, 102, 103], 'seed': 0}, {'missing_noise_rms': 31.5625, 'layer_sites_ix': [31, 62, 63, 94, 95, 125, 126, 127], 'seed': 0}, {'missing_noise_rms': 32.21875, 'layer_sites_ix': [22, 23, 24, 25, 26, 27, 28, 29, 30, 54, 55, 56, 57, 58, 59, 60, 61, 86, 87, 88, 89, 90, 91, 92, 93, 118, 119, 120, 121, 122, 123, 124], 'seed': 0}, {'missing_noise_rms': 32.6875, 'layer_sites_ix': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 32, 33, 34, 35, 36, 37, 38, 39, 

In [ ]:
# takes 1-2 mins

# - get max in vivo trace
PreRecording_h_vivo = si.load_extractor(PREP_PATH_h_vivo)
traces_vivo = PreRecording_h_vivo.get_traces()

# - get max in silico trace
PreRecording_h_silico = si.load_extractor(PREP_PATH_h_silico)
traces_silico = PreRecording_h_silico.get_traces()

In [ ]:
# takes 6 min

# scale and adjust background noise to match in vivo
ntimepoints = traces_silico.shape[0]

# scale traces
scaled_traces_silico = traces_silico * scale
scaled_with_noise = copy.copy(scaled_traces_silico)

for ix in range(len(layers_missing_noise)):
    # add missing noise
    sites = layers_missing_noise[ix]["layer_sites_ix"]
    nsites = len(sites)
    missing_noise_traces = np.random.normal(
        0, layers_missing_noise[ix]["missing_noise_rms"], [ntimepoints, nsites]
    )
    scaled_with_noise[:, sites] = scaled_traces_silico[:, sites] + missing_noise_traces

data9
data9
data9



In [46]:
contents

'data9\ndata9\n'